In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
#import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/.julia/dev/ALMPC/examples/Project.toml`
Precompiling project...
  ✓ HTTP
  ✓ GR
  ✓ Plots
  3 dependencies successfully precompiled in 50 seconds (246 already precompiled)


In [2]:
using ALMPC
using StaticArrays
using SparseArrays
using LinearAlgebra
using OSQP
using Plots
using RobotDynamics
using Test

┌ Info: Precompiling ALMPC [4af787c9-3890-4ef7-a76d-b0ed3d7a1470]
└ @ Base loading.jl:1342
┌ Warning: Package ALMPC does not have StaticArrays in its dependencies:
│ - If you have ALMPC checked out for development and have
│   added StaticArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ALMPC
└ Loading StaticArrays into ALMPC from project dependency, future warnings for ALMPC are suppressed.
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1342


## System dynamics ##

A 2D point mass $m=2\; \text{kg}$ in subject to an input force $F_{ext}$ and viscous friction with coefficient $b = 0.3\;[N \cdot \frac{s}{m}]$.

\begin{equation}
\begin{split}
\dot p &= v\\
\dot v &= -\frac{b}{m}v + \frac{1}{m}F_{ext}
\end{split}
\end{equation}

<span style='color:blue'> 


#### Continuous System Dynamics:

$$ \underbrace{\begin{bmatrix}
        \dot{x} \\
        \dot{y} \\
        \ddot{x} \\
        \ddot{y} 
\end{bmatrix}}_{\dot{x}} = \underbrace{\begin{bmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & -\frac{b}{m} & 0 \\        
        0 & 0 & 0 & -\frac{b}{m} 
\end{bmatrix}}_{\text{A}} * \underbrace{\begin{bmatrix}
        x\\
        y\\
        \dot{x}\\
        \dot{y} 
\end{bmatrix}}_{\text{x}} + \underbrace{\begin{bmatrix}
        0 & 0\\
        0 & 0\\
        \frac{1}{m} & 0 \\
        0 & \frac{1}{m} \\
\end{bmatrix}}_{\text{B}}* \underbrace{\begin{bmatrix}
        F_{x}\\
        F_{y}
\end{bmatrix}}_{\text{u}}$$
</span>


<span style='color:red'> 
    
#### Discrete System Dynamics:
$$ \underbrace{\begin{bmatrix}
        \dot{x} \\
        \dot{y} \\
        \ddot{x} \\
        \ddot{y} 
\end{bmatrix}}_{\dot{x}} = \underbrace{\begin{bmatrix}
        1 & 0 & Ts & 0 \\
        0 & 1 & 0 & Ts \\
        0 & 0 & 1-(\frac{b}{m}*T_{s}) & 0 \\        
        0 & 0 & 0 & 1-(\frac{b}{m}*T_{s}) 
\end{bmatrix}}_{\text{A}} * \underbrace{\begin{bmatrix}
        x\\
        y\\
        \dot{x}\\
        \dot{y} 
\end{bmatrix}}_{\text{x}} + \underbrace{\begin{bmatrix}
        0 & 0\\
        0 & 0\\
        \frac{1}{m}*T_{s} & 0 \\
        0 & \frac{1}{m}*T_{s} \\
\end{bmatrix}}_{\text{B}}* \underbrace{\begin{bmatrix}
        F_{x}\\
        F_{y}
\end{bmatrix}}_{\text{u}}$$

</span>


In [3]:
# Planar Point Mass Dynamics
function pointmass_dynamics(x,u; mass = 2, damp = 0.3)
    xdot = zero(x) 
    xdot[1] = x[3]
    xdot[2] = x[4]
    xdot[3] = -(damp/mass)*x[3] + u[1]/mass 
    xdot[4] = -(damp/mass)*x[4] + u[2]/mass 
    return xdot
end

# Classic RK4 integration with zero-order hold on u
function rk4(f,x,u,h)
    f1 = f(x, u)
    f2 = f(x + 0.5*h*f1, u)
    f3 = f(x + 0.5*h*f2, u)
    f4 = f(x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

# Designing a linear trajectory from x0 to xref
function nominal_trajectory(x0,N,dt)
    Xref = [zero(x0) for k = 1:N]
    # Design a trajectory that linearly interpolates from x0 to the origin
    for k = 1:N
        Xref[k][1:2] .= (N-k)/(N-1)*x0[1:2]
    end
    for k = 1:N-1
        Xref[k][3:4] .= (Xref[2][1:2] - Xref[1][1:2])/dt
    end
    return SVector{4}.(Xref)
end

nominal_trajectory (generic function with 1 method)

In [4]:
# Setting up the problem
dt = 0.05                # Time step [s]
Tfinal = 1           # Final time [s]
Nt = Int(Tfinal/dt)+1    # Number of time steps

Nx = 4  # Number of states
Nu = 2  # Number of controls

x0 = [3.0, 7.0, -3.0, -1.0]
# xfinal = [10.0, 10.0, 0.0, 0.0]
ueq = zeros(2)

# Generate reference trajectory
Xref = nominal_trajectory(x0,Nt,dt)     # Reference trajectory for all states
Uref = [copy(ueq) for k = 1:Nt]             # Reference inputs 
tref = range(0,Tfinal, length=Nt)           # Array of timesteps

mass = 2    # Mass [kg]
damp = 0.3  # Damping coefficient [N-s/m]

# Discretized MPC Model dynamics: x_k+1 = Ad*x_k + Bb*u_k
A = [1.0    0.0     dt                   0.0             ;
     0.0    1.0     0.0                 dt               ;
     0.0    0.0     1-(damp/mass)*dt     0.0             ;
     0.0    0.0     0.0                 1-(damp/mass)*dt]    # State Matrix
B = zeros(4, 2)                                             # Input Matrix
B[3,1] = (1/mass)*dt
B[4,2] = (1/mass)*dt

# #State Constraints
# xmin = [-100.0 -100.0]  
# xmax = [100.0 100.0]    

# #Control Constraints
# umin = [-0.5; -0.5]   
# umax = [0.5; 0.5]    

# # Constraints input variation with respect to previous sample
# Dumin = [-2e-1] 
# Dumax = [2e-1]

# MPC objective function weights
Q = Array(10.0*I(Nx));
R = Array(.01*I(Nu));
Qf = Array(10.0*I(Nx));

In [5]:
function get_k(controller, t) 
    searchsortedlast(controller.times, t)
end

function get_control(ctrl::ALMPC.MPCController{OSQP.Model}, x, time)
    # Update the QP
    ALMPC.updateQP!(ctrl, x, time)
    OSQP.update!(ctrl.solver, q=ctrl.q, l=ctrl.lb, u=ctrl.ub)

    # Solve QP
    results = OSQP.solve!(ctrl.solver)
    Δu = results.x[1:2]
    
    k = get_k(ctrl, time)
    return ctrl.Uref[k] + Δu 
end

function simulate(f, x0, ctrl; tf=2.0, dt=0.1, w=0.1)
    n = ctrl.Nx
    m = ctrl.Nu
    times = range(0, tf, step=dt)
    N = length(times)
    X = zeros(length(x0),N)
    U = [@SVector zeros(m) for k = 1:N-1]
    X[:,1] .= x0

    tstart = time_ns()
    for k = 1:N-1
        U[k] = get_control(ctrl, X[:,k], times[k]) 
        X[:,k+1] = rk4(f, X[:,k], U[k],dt)
    end
    tend = time_ns()
    rate = N / (tend - tstart) * 1e9
    println("Controller ran at $rate Hz")
    return X,U,times
end

simulate (generic function with 1 method)

In [6]:
Nmpc = 15           # MPC Horizon
mpc1 = ALMPC.OSQPController(Nx, Nu, Nmpc, Q, R, Qf,A, B, length(Xref))

# Provide the reference trajectory
mpc1.Xref .= Xref
mpc1.Uref .= Uref
mpc1.times .= tref

# Build the sparse QP matrices
ALMPC.buildQP!(mpc1, A,B,Q,R,Qf, tol=1e-6)

Xmpc1,Umpc1,tmpc1 = simulate(pointmass_dynamics, x0, mpc1, tf=Nmpc)

# @test norm(Xmpc1[end]) < 1e-6  
@show Xmpc1[:,end]

Controller ran at 752.7143302462043 Hz
Xmpc1[:, end] = [2.5383360955829636e-5, 0.00014034218681987265, -1.35995113394247e-5, -7.51904038388654e-5]


4-element Vector{Float64}:
  2.5383360955829636e-5
  0.00014034218681987265
 -1.35995113394247e-5
 -7.51904038388654e-5

In [7]:
# x position
#@show size(Xmpc1[1,:]), size(tref)
plot(tref, Xmpc1[1,:]) 

BoundsError: BoundsError: attempt to access 21-element StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}} at index [1:151]

In [ ]:
# y position
plot(tref, Xmpc1[2,:])

BoundsError: BoundsError: attempt to access 21-element StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}} at index [1:301]

In [8]:
# x, y Force
plot(tref[1:length(tref)-1], Umpc1)

LoadError: Expects 20 elements in each col of y, found 150.

In [ ]:
anim = @animate for i in 1:length(Xmpc1[1,:])
    scatter([Xmpc1[1,i]],
            [Xmpc1[2,i]], 
            m = (:circle, 6), 
            xlims = (0, 20),
            ylims = (0, 20),
            xlabel = "x position [m]",
            ylabel = "y position [m]")
    traj2!(Xref)
end every 10;
gif(anim, "state_ref_MPC.gif")